# Day 1-00.05: QLoRA 파인튜닝 개념 정리

이 노트북은 **실제 코드 대신 개념을 이해하는** 용도로 사용합니다. 핵심 아이디어만 빠르게 훑어보고, 실습은 main-practice/03 노트북에서 진행합니다.

### 1. LoRA 핵심 파라미터 정리
- `r`: 추가 행렬의 랭크. 클수록 표현력↑, 메모리·연산량↑
- `lora_alpha`: LoRA 가중치 스케일. 모델 본체와 어댑터의 균형 조절
- `target_modules`: LoRA 어댑터를 붙일 모듈(q/k/v/out 등)
- `lora_dropout`: 학습 중 일부 연결을 끊어 과적합 방지

In [ ]:
print("🔧 LoRA 파라미터 설명")
print("=" * 40)
print("1) r (rank): 작으면 빠르고 메모리 적게 사용, 크면 표현력↑")
print("2) lora_alpha: 어댑터 가중치를 얼마나 반영할지 결정")
print("3) target_modules: 어댑터를 붙일 레이어 목록")
print("4) lora_dropout: 드롭아웃 확률")

### 2. 4bit + LoRA를 쓰는 이유
- **4bit quantization**: 모델 본체를 압축해 Colab에서도 다룰 수 있도록 메모리를 절약합니다.
- **LoRA 어댑터**: 전체 가중치 대신 소수의 파라미터만 학습해 빠르고 가볍게 미세조정합니다.
- 이 노트북에서는 "어떤 이점이 있는지"를 표와 설명으로만 살펴본 뒤, 실전 코드는 main-practice/03에서 다룹니다.

### 3. QLoRA 코드 흐름 한눈에 보기 (실행하지 않음)
```
1) bitsandbytes 4bit 설정
2) AutoModel + AutoTokenizer 로드
3) LoraConfig → prepare_model_for_kbit_training → get_peft_model
4) 토크나이징 및 Trainer 구성
5) trainer.train() 실행
```
실제 코드는 main-practice/03 노트북에 준비되어 있으니 거기에서 실행하세요.

### 4. LoRA vs Full Fine-tuning 비교 표

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    "항목": [
        "메모리 사용량",
        "학습 시간",
        "학습하는 파라미터",
        "정확도",
        "재사용性",
    ],
    "Full Fine-tuning": [
        "수십 GB",
        "길다 (수 시간 이상)",
        "전체 모델",
        "높음",
        "낮음 (전체 모델 저장 필요)",
    ],
    "LoRA": [
        "수 GB 이하",
        "짧다 (수십 분~1시간)",
        "수백만 ~ 수천만",
        "높음 (기본 모델 대비 95% 수준)",
        "높음 (어댑터만 교체)",
    ]
})

print("⚖️ LoRA vs Full Fine-tuning")
print("=" * 40)
display(comparison)

### 5. QLoRA 파이프라인 요약

1. **데이터 준비** – 00.03 노트북에서 RAFT 템플릿으로 변환
2. **모델 준비** – 00.04 노트북에서 4bit + LoRA 어댑터 장착
3. **파인튜닝 실행** – main-practice/03 노트북에서 `trainer.train()` 수행
4. **평가** – 00.06 또는 main-practice/04 노트북에서 지표 확인

### 6. 다음 단계
- 개념 정리가 끝났다면 실제 파인튜닝 코드는 **main-practice/03_fine_tuning_with_lora.ipynb**에서 실행하세요.
- 변화 전후의 성능은 **00.06-evaluation.ipynb** 또는 main-practice/04 노트북에서 비교할 수 있습니다.